In [104]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd

In [105]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')

In [106]:
train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

train['Sex'] = train['Sex'].map({'male': 0, 'female': 1})

train['Embarked'] = train['Embarked'].fillna('S')
train['Embarked'] = train['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

train['Age'] = train['Age'].fillna(train['Age'].mean())


In [107]:
train.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [108]:
X = train.drop('Survived',axis=1)
y = train['Survived']

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 10)

In [110]:
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.long)

In [111]:
class TitanicDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return X, y

In [112]:
train_dataset = TitanicDataset(X_train, y_train)
test_dataset = TitanicDataset(X_test, y_test)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [113]:
class TitanicModel(nn.Module):
    def __init__(self):
        super(TitanicModel, self).__init__()
        self.fc1 = nn.Linear(7, 16)
        self.fc2 = nn.Linear(16, 32)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.softmax(self.fc3(x), dim=1)
        return x


In [114]:
model = TitanicModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [115]:
for epoch in range(23):
    train_loss = 0
    train_acc = 0
    for X, y in train_dataloader:
        optimizer.zero_grad()
        y_pred = model(X)
        loss = criterion(y_pred, y)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (y_pred.argmax(dim=1) == y).sum().item()
    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataset)
    
    # Оценка модели на тестовом наборе
    test_loss = 0
    test_acc = 0
    with torch.no_grad():
        for X, y in test_dataloader:
            y_pred = model(X)
            loss = criterion(y_pred, y)
            test_loss += loss.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
        test_loss /= len(test_dataloader)
        test_acc /= len(test_dataset)
    
    # Вывод результатов
    print(f'Epoch {epoch + 1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Test Loss={test_loss:.4f}, Test Acc={test_acc:.4f}')

Epoch 1: Train Loss=0.6671, Train Acc=0.5923, Test Loss=0.5635, Test Acc=0.7649
Epoch 2: Train Loss=0.6299, Train Acc=0.6677, Test Loss=0.5579, Test Acc=0.7724
Epoch 3: Train Loss=0.6256, Train Acc=0.6581, Test Loss=0.5612, Test Acc=0.7500
Epoch 4: Train Loss=0.6173, Train Acc=0.6838, Test Loss=0.5491, Test Acc=0.7761
Epoch 5: Train Loss=0.6151, Train Acc=0.6838, Test Loss=0.5439, Test Acc=0.7836
Epoch 6: Train Loss=0.6127, Train Acc=0.6790, Test Loss=0.5568, Test Acc=0.7687
Epoch 7: Train Loss=0.6172, Train Acc=0.6854, Test Loss=0.5402, Test Acc=0.7761
Epoch 8: Train Loss=0.6059, Train Acc=0.6966, Test Loss=0.5357, Test Acc=0.7724
Epoch 9: Train Loss=0.5984, Train Acc=0.6838, Test Loss=0.5373, Test Acc=0.7761
Epoch 10: Train Loss=0.5948, Train Acc=0.6918, Test Loss=0.5360, Test Acc=0.7761
Epoch 11: Train Loss=0.5891, Train Acc=0.6982, Test Loss=0.5331, Test Acc=0.7761
Epoch 12: Train Loss=0.5858, Train Acc=0.7127, Test Loss=0.5351, Test Acc=0.7575
Epoch 13: Train Loss=0.5847, Train Ac